
# Introduction to Swift and the Robotics Toolbox for Python

_Note: This notebook must be run locally. Due to how the Swift Simulator operates, this notebook will not run on Google Colab_

## Setup

First we must import the key packages which we will use throughout the exercises

In [1]:
import roboticstoolbox as rtb
import spatialmath as sm
import swift
import numpy as np
import spatialgeometry as sg

Swift is a browser based robot simulator. We must first make the environment and
then use the `launch` method to open the simulator.

Each time `launch()` is called, a new browser tab is opened with a clean Swift
environment.

By passing the `realtime=True` keyword argument to the `launch` method, the
simulator will try to run as close as possible to realtime, instead of as fast
as possible.

In [2]:
# Make the environment
env = swift.Swift()

# Launch the simulator, will open a browser tab
env.launch(realtime=True)

## Playing with a Robot

Now we can make a robot.

We can change the physical location of the Panda robot in the environment with the
`base` property. In this example, we move the Panda 1 metre along the y-axis.

Finally, we add the Panda to the environment using the `add()` method. Now you
should see a Panda loaded into the Swift environment, located 1 metre along the
y-axis.

Exercise 1
* Modify the code to position the robot at x = -0.1, y = 1.0, z = 0.0

In [3]:
# Make a Panda Robot
panda = rtb.models.Panda()

# Move the Panda along the y-axis
panda.base = sm.SE3.Ty(1.0)

# Add the Panda to the Swift environment
env.add(panda)

0

In this stage we are going to change the configuration of the Panda.

We can change the configuration of a robot using the `q` property.

After changing the robot state, we must use the `step(dt)` method to update the
swift environment. The `dt` parameter to the `step` method defines how much time
to step through.

Exercise 2
* Modify the code to change the joint angles of the robot. Make sure you step the environment to visualise the results

In [4]:
# Change the robot configuration
panda.q = [0, -0.3, 0, -2.2, 0, 2.0, np.pi / 4]

# Step the environment to see the updated robot
env.step(0)

We can also velocity control the robot using the `qd` property of the robot.

`qd` specifies the joint velocities $\dot{q}$ for each joint within the robot.

In this example we are going to actuate the first and fifth joint of the robot
at 0.1 rad/s.

We are then going to simulate the robot for 5 seconds, stepping the environment
in 10 ms increments. As the robot is simulated, the robot's configuration is
updated in the `q` property.

Exercise 3
* Modify the code so that the robot's second joint rotates at 0.2 rad/s

In [5]:
# Set the joint velocities of the robot
panda.qd = [0.1, 0, 0, 0, -0.1, 0, 0]

# Set the simulation step size to 10 ms
dt = 0.1

# To simulate for 5 seconds, we need 50 steps
for _ in range(50):
    env.step(dt)

# Print the new robot joint configuration
print(panda.q)

[ 0.5        -0.3         0.         -2.2        -0.5         2.          0.78539816]


End-effector pose

To find out where the end-effector is located in the world we use forward
kinematics. This can be calculated in the toolbox using the `fkine(q)` method.
The argument `q` species to robot configuration in which to calculate the
forward kinematics.

Note that the forward kinematics includes the base transform. We can choose to
ignore the base transform by passing the `include_base=False` keyword argument.

Exercise 4
* Calculate and print the forward kinematics of the robot at the joint configuration `[0.1, 0.0, -1.1, 0.5, 0.3, -0.3, 0.9]`

In [6]:
# Work out the forward kinemtics of the robot at the zero-angle configuration
print("Forward kinematics at zero-angle configuration")
print(panda.fkine([0, 0, 0, 0, 0, 0, 0]))

# Work out the forward kinemtics of the robot at the current configuration
print("Forward kinematics at current configuration")
print(panda.fkine(panda.q))

# Work out the transform between the base and end-effector of the robot at
# the current configuration
print("Robot base to end-effector transform at current configuration")
print(panda.fkine(panda.q, include_base=False))


Forward kinematics at zero-angle configuration
   0.7071    0.7071    0         0.088     
   0.7071   -0.7071    0         1         
   0         0        -1         0.8226    
   0         0         0         1         

Forward kinematics at current configuration
   0.7631    0.5568    0.3282    0.4657    
   0.6442   -0.6958   -0.3175    1.17      
   0.05163   0.4537   -0.8897    0.4306    
   0         0         0         1         

Robot base to end-effector transform at current configuration
   0.7631    0.5568    0.3282    0.4657    
   0.6442   -0.6958   -0.3175    0.1699    
   0.05163   0.4537   -0.8897    0.4306    
   0         0         0         1         



## Resolved-Rate Motion Control

We use the manipulator Jacobian $\mathbf{J}(q)$ to control the robot's end-effector spatial velocity $^e\nu$ using the joint velocities $\dot{q}$.

The end-effector spatial velocity \
\
$\nu = (v_x, v_y, v_z, \omega_x, \omega_y, \omega_z)^\top$ \
\
is related to joint velocities by \
\
$^e\nu = \mathbf{J}(q) \dot{q}$

We can use the inverse of this relationship to specify a desired end-effector velocity $^e\nu$ and get the required joint velocities $\dot{q}$ with \
\
$\dot{q} = \mathbf{J}{(q)^+} \ ^e\nu $ \
\
where Moore-Penrose Pseudoinverse $(\cdot)^+$ is used if the Jacobian is not square (the number of robot joints $n$ is greater than 7).


In [7]:
# Change the robot configuration to the ready position
panda.q = [0, -0.3, 0, -2.2, 0, 2.0, np.pi / 4]

# Specify our desired end-effector velocity
ev = [0.05, 0, 0, 0, 0, 0]

# Run the simulation for 5 seconds
for _ in range(50):

    # Work out the manipulator Jacobian using the current robot configuration
    J = panda.jacobe(panda.q)

    # Since the Panda has 7 joints, the Jacobian is not square, therefore we must
    # use the pseudoinverse (the pinv method)
    J_inv = np.linalg.pinv(J)

    # Calculate the required joint velocities and apply to the robot
    panda.qd = J_inv @ ev

    # Step the simulator by dt seconds
    env.step(dt)

### Reference frames

An important point to note here is that the reference frame matters.

The end-effector spatial velocity $\nu$ could be in any reference frame.

However, most commonly, the it would be specified in the end-effector frame.

In which case we specify it as $^e\nu$ (where the $^e$ indicates it is in the end-effector frame).

Alternatively, we could specify it in the base frame of the robot. We specify it as $^0\nu$ in this case.

The manipulator Jacobian also has a reference frame. The toolbox can calculate both the end-effector frame Jacobian 

$^e\mathbf{J}(q)$ - use the toolbox method `jacobe`

and the base frame manipulator Jacobian

$^0\mathbf{J}(q)$ - use the toolbox method `jacob0`.

The Jacobian frame must match the velocity frame or you will get unexpected results.


In the following example we are going to illustrate the difference between using $^e\mathbf{J}(q)$ and $^0\mathbf{J}(q)$.

In the first case we are going to give make the end-effector move at 0.05 m/s along the x-axis of the base frame

Therefore we are specifying  base frame velocity $^0\nu = (0.05, 0, 0, 0, 0, 0)^\top$

Since we specified a base-frame velocity we must use the base frame Jacobian $^0\mathbf{J}(q)$. 

Exercise 5
* Modify the code so the robot rotates around the z-axis of the base frame

In [8]:
# Change the robot configuration
panda.q = [0, -1.1, 0, -2.1, 0, 2.0, np.pi / 4]
env.step()

# Specify our desired end-effector velocity
ev = [0.05, 0, 0, 0, 0, 0]

# Run the simulation for 5 seconds
for _ in range(50):

    # Work out the base frame manipulator Jacobian using the current robot configuration
    J = panda.jacob0(panda.q)

    # Since the Panda has 7 joints, the Jacobian is not square, therefore we must
    # use the pseudoinverse (the pinv method)
    J_inv = np.linalg.pinv(J)

    # Calculate the required joint velocities and apply to the robot
    panda.qd = J_inv @ ev

    # Step the simulator by dt seconds
    env.step(dt)

Now we will do the same thing but with the end-effector frame Jacobian $^e\mathbf{J}(q)$.

Note how the robot now moves along the x-axis of the end-effector frame rather than the base frame.

Exercise 6
* Modify the code so the robot rotates around the z-axis of the end-effector frame

In [9]:
# Change the robot configuration
panda.q = [0, -1.1, 0, -2.1, 0, 2.0, np.pi / 4]
env.step()

# Specify our desired end-effector velocity
ev = [0.05, 0, 0, 0, 0, 0]

# Run the simulation for 5 seconds
for _ in range(50):

    # Work out the end-effector frame manipulator Jacobian using the current robot configuration
    J = panda.jacobe(panda.q)

    # Since the Panda has 7 joints, the Jacobian is not square, therefore we must
    # use the pseudoinverse (the pinv method)
    J_inv = np.linalg.pinv(J)

    # Calculate the required joint velocities and apply to the robot
    panda.qd = J_inv @ ev

    # Step the simulator by dt seconds
    env.step(dt)

### Closing the Loop

We want the robot to go to a specific location in the world. Before we do that, lets add some objects to the world so we can visualise the desired destinations.

We will also add an axes to visualise the orientation of the end-effector

We can make objects using the `spatialgeometry` package. I this case we are going to add an axes, a cube, and a sphere.


Exercise 7
* Add another cuboid to the environment
    * The cuboid has a length of 0.2m, width of 0.4m and a height of 0.8m
    * The color needs to be green with an opacity of 30%
    * Make the cuboid located at x = -1.0, y = 0.0, z = 0.5

In [10]:
# Change the robot configuration to the ready position
panda.q = [0, -0.3, 0, -2.2, 0, 2.0, np.pi / 4]
env.step(0)

# Make the end-effector axes, with a length of 10cm
# Make the end-effector axes located at the end-effector
ee_ax = sg.Axes(0.1, base=panda.fkine(panda.q))

# Make the goal axes, with a length of 10cm
# Make the goal axes located at x=0.5m, y=0.6m and z=0.5m
goal_ax = sg.Axes(0.1, base=sm.SE3(0.5, 0.6, 0.5) * sm.SE3.Rx(np.pi))

# Make the cuboid, with a length, width and height of 10cm
# Make the cuboid located at x=0.5m, y=1m and z=0.05m
# Make the color red with an opacity of 50%
goal_cube = sg.Cuboid(
    [0.1, 0.1, 0.1],
    base=sm.SE3(0.5, 1, 0.05) * sm.SE3.Rx(np.pi),
    color=[1, 0.2, 0.1, 0.5]
)

# Make the sphere, with a radius of 5cm
# Make the sphere located at x=0.5m, y=1.3m and z=0.3m
# Make the color blue with an opacity of 100%
goal_sphere = sg.Sphere(
    0.05,
    base=sm.SE3(0.5, 1.3, 0.3) * sm.SE3.Rx(np.pi),
    color=[0.0, 0.2, 1, 1.0]
)

# Add the objects to the environment
env.add(ee_ax)
env.add(goal_ax)
env.add(goal_cube)
env.add(goal_sphere)


4


To get the robot to go a location in the world, we must update the desired end-effector spatial velocity at each time step.

To do this, we set the end-effector spatial velocity to the spatial error between the end-effectors current pose and the goal pose.

We can do this easily using the `p_servo` method in the toolbox. Lets try to make the robot reach the axes first.

`p_servo` takes
* the current robot pose, which we can calculate using the `panda.fkine` method
* the goal pose which we can access using the `goal_ax.base` attribute
* the gain, which adjusts how quickly the robot will reach the goal
* the error threshold, which specifies a minimum distance before the robot has arrived

`p_servo` returns
* the spatial error multiplied by the gain which we can use as $^e\nu$
* arrived, a boolean specifying if the robot has arrived or not

Exercise 8
* Print the spatial error to the cuboid `goal_cube`
* Print the spatial error to the sphere `goal_sphere`

In [11]:
# Calculate the spatial error to the goal pose
error, arrived = rtb.p_servo(panda.fkine(panda.q), goal_ax.base, gain=1.0, threshold=0.001)
print(error, arrived)

[ 2.45959532e-02  4.00000000e-01 -8.53410764e-02 -1.53106918e-18 -1.00000000e-01 -5.26627616e-17] False


To make the robot approach the goal, we introduce a loop, and we re-calculate the required end-effector spatial velocity on each step.

Exercise 9
* Make the robot reach for the cuboid `goal_cube`
* Make the robot reach for the sphere `goal_sphere`
* Make the robot reach for the axes `goal_ax` twice as fast
* Make the robot reach for the cuboid `goal_cube` four times faster
* Make the robot reach for the sphere `goal_sphere` twice as slow

In [12]:
# Change the robot configuration to the ready position
panda.q = [0, -0.3, 0, -2.2, 0, 2.0, np.pi / 4]
env.step(0)

# A variable to specify when to break the loop
arrived = False

dt = 0.05

# Run the simulation until the robot arrives at the goal
while not arrived:

    # Set the pose of ee_ax to be the pose of the end-effector
    ee_ax.base = panda.fkine(panda.q)

    # Work out the end-effector frame manipulator Jacobian using the current robot configuration
    J = panda.jacobe(panda.q)

    # Since the Panda has 7 joints, the Jacobian is not square, therefore we must
    # use the pseudoinverse (the pinv method)
    J_inv = np.linalg.pinv(J)

    # Calculate the required end-effector velocity and whether the robot has arrived
    ev, arrived = rtb.p_servo(panda.fkine(panda.q), goal_ax.base, gain=1.0, threshold=0.001)

    # Calculate the required joint velocities and apply to the robot
    panda.qd = J_inv @ ev

    # Step the simulator by dt seconds
    env.step(dt)